# Delete metadata in PANCAN_TWIST_Dev

In [204]:
from firecloud_sugar import *
import warnings
import glob
import numpy as np
import dalmatian as dog
import pandas as pd
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [205]:
namespace = "nci-mimoun-bi-org"
workspace = "PANCAN_TWIST_Dev"

namespace_gp = "broad-genomics-delivery"
workspace_gp = "Cancer_Cell_Line_Factory_CCLF_PanCancer_PanelSeq"

In [206]:
wm: dog.WorkspaceManager
wm = dog.WorkspaceManager(namespace,workspace)

wm_gp: dog.WorkspaceManager
wm_gp = dog.WorkspaceManager(namespace_gp, workspace_gp)

In [ ]:
# Delete all sample_sets
all_sample_sets = wm.get_sample_sets()
[wm.delete_sample_set(sample_set_index) for sample_set_index in all_sample_sets.index]

In [ ]:
# Delete all samples
all_samples = wm.get_samples()
[wm.delete_sample(sample_index) for sample_index in all_samples.index]

In [ ]:
# Delete all the participants
all_participants = wm.get_participants()
[wm.delete_participant(participant_index) for participant_index in all_participants.index]

In [24]:
# Upload participant metadata
twist2_external = pd.read_excel("twist_external_sample_ids/TWIST2_External_Sample_ID.xlsx")
collaborator_participant_id = twist2_external['Collaborator Participant ID']

In [33]:
unique_participant_ids = collaborator_participant_id.unique()
#unique_participant_ids = np.insert(unique_participant_ids, 0, 'entity:participant_id')
unique_participant_ids

array(['CCLF_AB1065', 'CCLF_AB1097', 'CCLF_BU1013', 'CCLF_BU1017',
       'CCLF_cRCRF1040', 'CCLF_cRCRF1060', 'CCLF_cRCRF1081',
       'CCLF_cRCRF1085', 'CCLF_CY1006', 'CCLF_CY1007', 'CCLF_CY1015',
       'CCLF_KL1235', 'CCLF_KL1282', 'CCLF_KL1283', 'CCLF_KL1288',
       'CCLF_KL1292', 'CCLF_KL1294', 'CCLF_PEDS1064', 'CCLF_PEDS1140',
       'CCLF_PEDS1141', 'CCLF_PEDS1143', 'CCLF_PEDS1153', 'CCLF_PEDS1154',
       'CCLF_PEDS1155', 'CCLF_PEDS1156', 'CCLF_RCRF1041', 'CCLF_RCRF1064',
       'CCLF_RCRF1070', 'CCLF_RCRF1097', 'CCLF_RCRF1099', 'CCLF_RCRF1102',
       'CCLF_SS1020', 'CCLF_CTRL', 'SP014'], dtype=object)

In [38]:
# unique_participant_fc = pd.DataFrame(unique_participant_ids)
wm.upload_participants(unique_participant_ids)

Successfully imported 34 participants.


In [231]:
# Get sample set Twist 2
gp_sample_sets = wm_gp.get_sample_sets()
twist2_gp_sample_sets = gp_sample_sets[gp_sample_sets.index == 'CCLF_TWIST2']

In [232]:
all_twist2_samples = twist2_gp_sample_sets['samples'].values[0]

In [233]:
# Get all samples
gp_samples = wm_gp.get_samples()

In [234]:
only_twist2_samples = gp_samples[gp_samples.index.isin(all_twist2_samples)]

In [235]:
only_twist2_samples.index.values[1:]

array(['RP-571_PDO-18546_SM-J1OYQ_CCLF_cRCRF1040T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OYR_CCLF_BU1017N_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OYS_CCLF_cRCRF1060T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OYT_CCLF_AB1065T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OYU_CCLF_KL1288T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OYV_CCLF_KL1283T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OYW_CCLF_cRCRF1040T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OYX_CCLF_KL1282T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OYY_CCLF_BU1013T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OYZ_CCLF_cRCRF1060T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OZ1_CCLF_cRCRF1060T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OZ2_CCLF_cRCRF1060T_v1_Custom_Selection_OnPrem',
       'RP-571_PDO-18546_SM-J1OZ3_CCLF_PEDS1143N_v1_Custom_Selection_O

In [237]:
# Extract SM-IDS from sample_ids
import re
only_twist2_samples.index

gp_ids = only_twist2_samples.index.values

for gp_id in only_twist2_samples.index.values[1:]:
    
    search_sm_id = re.search('(SM-[^_]*)_', gp_id)
    current_sm_id = search_sm_id.group(1)
    
    only_twist2_samples.at[gp_id, 'sm_id'] = current_sm_id

ctrl_series = twist2_external[twist2_external['Parent Sample ID'] == 'Ctrl']
ctrl_sm_id = ctrl_series['Exported DNA SM-ID']





#newlist = list(filter(r.match, mylist))
#sm_ids = re.match('')

# For the NA ctrl, fetch the SM-ID in the external sheet (Twist 2 in this case)

In [218]:
only_twist2_samples.iloc[0, 'sm_id'] = ctrl_sm_id.iloc[0]

In [246]:
only_twist2_samples.ix[0, 'sm_id'] = ctrl_sm_id.iloc[0]

In [247]:
only_twist2_samples

,rna_ribosomal_bases,wgs_pct_80x,hs_zero_cvg_targets_pct,min_insert_size,library-1_read_pair_duplicates,hs_fold_enrichment,window_size,wgs_pct_100x,rna_intergenic_bases,participant,...,hs_pct_exc_off_target,mean_insert_size,pct_contamination,num_in_db_snp,mean_read_length,wgs_pct_exc_baseq,rna_num_r1_transcript_strand_reads,wgs_pct_1x,hs_min_target_coverage,sm_id
sample_id,,,,,,,,,,,,,,,,,,,,,
RP-1872_NA12878_LCSET-15778_v1_Custom_Selection_OnPrem,null,0,.003949,2,1631804,1091.38773,0,0,0,RP-1872_NA12878_LCSET-15778,...,.208757,186.15159,null,0,146,0,0,0,NaN,SM-IV4IP
RP-571_PDO-18546_SM-J1OYQ_CCLF_cRCRF1040T_v1_Custom_Selection_OnPrem,null,0,.003723,3,1913209,1049.76278,0,0,0,RP-571_PDO-18546_SM-J1OYQ_CCLF_cRCRF1040,...,.21427,187.12407,null,0,146,0,0,0,NaN,SM-J1OYQ
RP-571_PDO-18546_SM-J1OYR_CCLF_BU1017N_v1_Custom_Selection_OnPrem,null,0,.000903,4,1837457,1099.90596,0,0,0,RP-571_PDO-18546_SM-J1OYR_CCLF_BU1017,...,.191004,184.51444,null,0,146,0,0,0,NaN,SM-J1OYR
RP-571_PDO-18546_SM-J1OYS_CCLF_cRCRF1060T_v1_Custom_Selection_OnPrem,null,0,.003723,9,1444012,1053.41249,0,0,0,RP-571_PDO-18546_SM-J1OYS_CCLF_cRCRF1060,...,.219469,185.28607,null,0,146,0,0,0,NaN,SM-J1OYS
RP-571_PDO-18546_SM-J1OYT_CCLF_AB1065T_v1_Custom_Selection_OnPrem,null,0,.990974,19,10633,1226.49865,0,0,0,RP-571_PDO-18546_SM-J1OYT_CCLF_AB1065,...,0,194.64534,null,0,146,0,0,0,NaN,SM-J1OYT
RP-571_PDO-18546_SM-J1OYU_CCLF_KL1288T_v1_Custom_Selection_OnPrem,null,0,.001128,2,1792445,1104.53829,0,0,0,RP-571_PDO-18546_SM-J1OYU_CCLF_KL1288,...,.18568,180.11934,null,0,146,0,0,0,NaN,SM-J1OYU
RP-571_PDO-18546_SM-J1OYV_CCLF_KL1283T_v1_Custom_Selection_OnPrem,null,0,.001128,2,1975391,1099.87591,0,0,0,RP-571_PDO-18546_SM-J1OYV_CCLF_KL1283,...,.178321,174.972,null,0,146,0,0,0,NaN,SM-J1OYV
RP-571_PDO-18546_SM-J1OYW_CCLF_cRCRF1040T_v1_Custom_Selection_OnPrem,null,0,.003949,9,1692915,1088.40188,0,0,0,RP-571_PDO-18546_SM-J1OYW_CCLF_cRCRF1040,...,.207211,186.93954,null,0,146,0,0,0,NaN,SM-J1OYW
RP-571_PDO-18546_SM-J1OYX_CCLF_KL1282T_v1_Custom_Selection_OnPrem,null,0,.003949,2,1673472,1094.31857,0,0,0,RP-571_PDO-18546_SM-J1OYX_CCLF_KL1282,...,.188643,178.79477,null,0,146,0,0,0,NaN,SM-J1OYX


In [228]:
only_twist2_samples.iloc[0]

rna_ribosomal_bases                      null
wgs_pct_80x                                 0
hs_zero_cvg_targets_pct               .003949
min_insert_size                             2
library-1_read_pair_duplicates        1631804
                                       ...   
wgs_pct_exc_baseq                           0
rna_num_r1_transcript_strand_reads          0
wgs_pct_1x                                  0
hs_min_target_coverage                    NaN
sm_id                                     NaN
Name: RP-1872_NA12878_LCSET-15778_v1_Custom_Selection_OnPrem, Length: 222, dtype: object

sample_id
RP-1872_NA12878_LCSET-15778_v1_Custom_Selection_OnPrem                   RP-1872_NA12878_LCSET-15778_v1_Custom_Selectio...
RP-571_PDO-18546_SM-J1OYQ_CCLF_cRCRF1040T_v1_Custom_Selection_OnPrem     RP-571_PDO-18546_SM-J1OYQ_CCLF_cRCRF1040T_v1_C...
RP-571_PDO-18546_SM-J1OYR_CCLF_BU1017N_v1_Custom_Selection_OnPrem        RP-571_PDO-18546_SM-J1OYR_CCLF_BU1017N_v1_Cust...
RP-571_PDO-18546_SM-J1OYS_CCLF_cRCRF1060T_v1_Custom_Selection_OnPrem     RP-571_PDO-18546_SM-J1OYS_CCLF_cRCRF1060T_v1_C...
RP-571_PDO-18546_SM-J1OYT_CCLF_AB1065T_v1_Custom_Selection_OnPrem        RP-571_PDO-18546_SM-J1OYT_CCLF_AB1065T_v1_Cust...
RP-571_PDO-18546_SM-J1OYU_CCLF_KL1288T_v1_Custom_Selection_OnPrem        RP-571_PDO-18546_SM-J1OYU_CCLF_KL1288T_v1_Cust...
RP-571_PDO-18546_SM-J1OYV_CCLF_KL1283T_v1_Custom_Selection_OnPrem        RP-571_PDO-18546_SM-J1OYV_CCLF_KL1283T_v1_Cust...
RP-571_PDO-18546_SM-J1OYW_CCLF_cRCRF1040T_v1_Custom_Selection_OnPrem     RP-571_PDO-18546_SM-J1OYW_CCLF_cRCRF1040T_v1_C...
RP-571